---

# **Master in Big Data Analytics**
# **Internet Contents Distribution**

# **LAB 1 APIs - WORLD BANK**

Authors: 
   - Yihang Zhu
   - Pablo Cano Turnes
   - Aleksandr Veretelnik
   - Miguel Rodríguez Losada


---

# 0. LAB PREPARATION

Students have to complete the following tasks before attending the lab:

1. **Read and study the API documentation to have some initial notions of the functionality of the World Bank API. Following, we share several links to the documentation related to the World Bank API:**
   - https://datahelpdesk.worldbank.org/knowledgebase/articles/898581-api-basic-call-structures  
   - https://datahelpdesk.worldbank.org/knowledgebase/topics/125589-developer-information  
   - https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-about-the-indicators-api-documentation  

2. **The key element of the World Bank API are the "indicators". Next, we share a link that may simplify the search of indicators through a search tool. Once you have selected an indicator you can find its codification within the url bar of the browser.**

   - https://data.worldbank.org/indicator?tab=featured

---

# 1. INTRODUCTION

* The goal of this lab is to gain experience testing a widely-used API such as the World Bank API that includes bunch of information about countries indicators in economy, health, education, agriculture, etc.

* The lab includes 5 milestones that will drive the student through the use of several indicators.

* The lab will be done in groups of 4 students.

* The lab will use two complete consecutive sessions (4 hours). The students are expected to complete the 5 milestones proposed in the lab within these 2 sessions.

* **The final mark will be computed as a function of the number of milestones successfully completed.**

* **Each group should also upload their lab notebook in the corresponding task in Aula Global.**

* Upon completing all the milestones, students should call the professor, who will check the correctness of the solution. Partial milestones checks may be allowed in some cases.

---

# 2. **MILESTONES**

In this section we describe one by one the milestones and leave a space to the students to implement the code to complete the requested task.

**NOTE: Unless otherwise stated, all the milestones have to deliver information about countries. Therefore, you should not consider regions or any other aggregated information in your analysis.**

---


## **2.1. MILESTONE 1: POPULATION**

Retrieve the 2022 countries' population and show:

- the **Top 10** countries, and  
- the **Bottom 10** countries  

within the World Bank database.

*Write your code solution for Milestone 1 in the next code cell in the notebook.*

---

In [1]:
# We load the necessary libraries

import requests
import pandas as pd

In [2]:
# SOLUTION MILESTONE 1

# We can face the following problem: The World Bank API returns not only real countries but also 'constructed' countries (Aggregates) such as "World", "High income", "Euro area", etc.
# https://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL?date=2022&format=json&per_page=20000 is not returning only real countries.

# First, we need to get the country metadata (Indicator API)
country_url = "https://api.worldbank.org/v2/country?format=json&per_page=3000"
country_response = requests.get(country_url).json()
country_metadata = country_response[1] 

# The reason we are selecting country_response[1] is that the World Bank API returns a two-element list:
# The first element (country_response[0]) contains metadata about the response (page number, total pages, etc.)
# The second element (country_response[1]) contains the actual list of countries and their metadata.

# Create a set of valid country codes (filtering out Aggregates)
valid_countries = []
for entry in country_metadata:
    # The key difference between real country and 'constructed' country: Aggregates have a region value of "Aggregates" 
    if entry['region']['value'] != 'Aggregates' :
        valid_countries.append(entry['id']) # We're storing the country codes (ISO3)

# Define a filter function to filter out non-country entries
def get_valid_country(entry):
    filtered_entry = []
    for item in entry:
        # The indicator response has a 'countryiso3code' field, or you can use item['country']['id']
        # We check if this code exists in our 'valid_countries' list
        if item['countryiso3code'] in valid_countries:
            filtered_entry.append(item)
    return filtered_entry

# Get the Raw Population Data
pop_url = "https://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL"
pop_params = {
    "format": "json",
    "per_page": 20000,
    "date": "2022"
}

pop_response = requests.get(pop_url, params=pop_params)
raw_pop_data = (pop_response.json())[1]

# Filter the Population JSON Data: Take only the real countries
filtered_pop_data = get_valid_country(raw_pop_data)

# Now, we can convert the filtered data into a pandas DataFrame
df = pd.json_normalize(filtered_pop_data)
df["value"] = pd.to_numeric(df["value"], errors="coerce") # Convert the 'value' column to numeric, coercing errors to NaN
df = df.dropna(subset=["value"]) # Drop rows where 'value' is NaN

# Let's select and rename the relevant columns
df = df[["country.value", "countryiso3code", "value"]]
df = df.rename(columns={"country.value": "country", "value": "population"})
df["population"] = df["population"].astype("int64")

# Let's display the top 10 and bottom 10 countries by population

## First we creare the objects
top10 = df.sort_values("population", ascending=False).head(10)
bottom10 = df.sort_values("population", ascending=True).head(10)

## Then, we display the objects
print("TOP 10 countries by population in 2022")
display(top10)

print("\nBOTTOM 10 countries by population in 2022")
display(bottom10)

TOP 10 countries by population in 2022


,country,countryiso3code,population
89,India,IND,1425423212
41,China,CHN,1412175000
206,United States,USA,334017321
90,Indonesia,IDN,278830529
149,Pakistan,PAK,243700667
144,Nigeria,NGA,223150896
26,Brazil,BRA,210306415
15,Bangladesh,BGD,169384897
161,Russian Federation,RUS,144236933
127,Mexico,MEX,128613117



BOTTOM 10 countries by population in 2022


,country,countryiso3code,population
201,Tuvalu,TUV,9992
137,Nauru,NRU,11801
150,Palau,PLW,17759
183,St. Martin (French part),MAF,28870
164,San Marino,SMR,33755
75,Gibraltar,GIB,37609
27,British Virgin Islands,VGB,38319
130,Monaco,MCO,38931
114,Liechtenstein,LIE,39493
124,Marshall Islands,MHL,40077


## **2.2. MILESTONE 2: WOMEN Vs. MEN POPULATION**

Obtain the **% of men and women** for each country in **2022** and compute the difference among them using the formula:

> **%women − %men**

Then display:

1. The number of countries with **more women than men**.  
2. The number of countries with **more men than women**.  
3. The **10 countries with more women** compared to men (ten countries with the largest positive value of the previous metric).  
4. The **10 countries with more men** compared to women (ten countries with the largest negative value of the previous metric).  

**Note**: You can use the indicators of the **absolute number of men and women** from the World Bank API and compute the percentages for each country and the difference, or you can use the indicators that give directly the percentages.

*Write your code solution for Milestone 2 in the next code cell in the notebook.*

---

In [3]:
# SOLUTION MILESTONE 2

# Procedure to get the women population percentage in 2022
url_female = "https://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL.FE.ZS"
params = {
    "format": "json", 
    "per_page": 20000, 
    "date": "2022"
    }

resp_f = requests.get(url_female, params=params) # Getting the responses from the API
data_f = resp_f.json() # Transforming the response to JSON format
female = pd.json_normalize(get_valid_country(data_f[1])) # Here we are filtering only real countries and creating the DataFrame

# Here, we're curating our dataframe for women percentage population
female["value"] = pd.to_numeric(female["value"], errors="coerce") # Convert the 'value' column to numeric, coercing errors to NaN
female = female.dropna(subset=["value"]) # We're getting rid of NaN values

# Here, we select and rename the relevant columns
female = female[["country.value", "countryiso3code", "value"]]
female = female.rename(columns={
    "country.value": "country",
    "value": "female_percent"
})


# Procedure to get the men population percentage in 2022

url_male = "https://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL.MA.ZS"
resp_m = requests.get(url_male, params=params) # Getting the responses from the API

data_m = resp_m.json() # Transforming the response to JSON format
male = pd.json_normalize(get_valid_country(data_m[1])) # Here we are filtering only real countries and creating the DataFrame

# Here, we're curating our dataframe for man percentage population
male["value"] = pd.to_numeric(male["value"], errors="coerce") # Convert the 'value' column to numeric, coercing errors to NaN
male = male.dropna(subset=["value"]) # We're getting rid of NaN values

# Then, we select and rename the relevant columns
male = male[["countryiso3code", "value"]]
male = male.rename(columns={"value": "male_percent"})

# Let's merge both dataframes to analyze the differences
gender = pd.merge(female, male, on="countryiso3code", how="inner")

# We compute the difference between women population percentage and men population percentage
gender["diff_women_men"] = gender["female_percent"] - gender["male_percent"]
gender = gender.dropna(subset=["female_percent", "male_percent", "diff_women_men"]) # Just in case, we drop NaN values

# Let's now answer the milestione questions

# 1 and 2: How many countries have a higher percentage of women/men
more_women = (gender["diff_women_men"] > 0).sum()
more_men = (gender["diff_women_men"] < 0).sum()

print("Number of countries with more women than men:", more_women)
print("Number of countries with more men than women:", more_men)

# 3: Top 10 with the highest differences in women-men population percentage

top10_more_women = gender.sort_values("diff_women_men", ascending=False).head(10)
print("\nTop 10 countries with more women than men:")
display(top10_more_women[["country", "countryiso3code", "female_percent", "male_percent", "diff_women_men"]])

# 3: Top 10 with the highest differences in women-men population percentage

top10_more_men = gender.sort_values("diff_women_men", ascending=True).head(10)
print("\nTop 10 countries with more men than women:")
display(top10_more_men[["country", "countryiso3code", "female_percent", "male_percent", "diff_women_men"]])


Number of countries with more women than men: 139
Number of countries with more men than women: 78

Top 10 countries with more women than men:


,country,countryiso3code,female_percent,male_percent,diff_women_men
86,"Hong Kong SAR, China",HKG,54.890124,45.109876,9.780248
129,Moldova,MDA,53.960481,46.039519,7.920963
117,"Macao SAR, China",MAC,53.749590,46.250410,7.499180
109,Latvia,LVA,53.718775,46.281225,7.437550
8,Armenia,ARM,53.696741,46.303259,7.393483
161,Russian Federation,RUS,53.544081,46.455919,7.088162
203,Ukraine,UKR,53.427489,46.572511,6.854978
72,Georgia,GEO,53.408415,46.591585,6.816831
17,Belarus,BLR,53.404022,46.595978,6.808045
183,St. Martin (French part),MAF,53.120887,46.879113,6.241773



Top 10 countries with more men than women:


,country,countryiso3code,female_percent,male_percent,diff_women_men
159,Qatar,QAT,28.152936,71.847064,-43.694128
204,United Arab Emirates,ARE,35.896864,64.103136,-28.206273
121,Maldives,MDV,37.763281,62.236719,-24.473438
14,Bahrain,BHR,38.027126,61.972874,-23.945749
148,Oman,OMN,38.444802,61.555198,-23.110397
106,Kuwait,KWT,38.945552,61.054448,-22.108896
166,Saudi Arabia,SAU,39.265643,60.734357,-21.468713
169,Seychelles,SYC,44.914836,55.085164,-10.170329
150,Palau,PLW,46.061152,53.938848,-7.877696
22,Bhutan,BTN,46.448649,53.551351,-7.102703


## **2.3. MILESTONE 3: GDP PER CAPITA ACCORDING TO INCOME LEVEL GROUPS**

Compute the **average increase/decrease in percentage** for the **GDP per capita in US dollars** in the following two periods:

- **2000–2022**  
- **2010–2022**

for the following **income groups**:

- low-income economies  
- lower-middle-income economies  
- middle-income economies  
- upper-middle-income economies  
- high-income economies  

The following link provides information on the different country aggregations carried out by the World Bank:  

https://datahelpdesk.worldbank.org/knowledgebase/articles/906519-world-bank-country-and-lending-groups

You should compute the %GDP increase as follows.  
Given a country A with a GDP per capita of \$20,000 in 2000 and \$30,000 in 2022, the increase/decrease should be computed as:

> %GDP increase = 100 × (30000 − 20000) / 20000 = 50%.

*Write your code solution for Milestone 3 in the next code cell in the notebook.*

---

In [4]:
# SOLUTION MILESTONE 3

# Procedure to get GDP per capita for years 2000, 2010, and 2022

## GPD per capita 2000

url_gdp = "https://api.worldbank.org/v2/country/all/indicator/NY.GDP.PCAP.CD"

params_2000 = {
    "format": "json", 
    "per_page": 20000, 
    "date": "2000"}

resp_2000 = requests.get(url_gdp, params=params_2000) # Getting the response from the API
data_2000 = resp_2000.json()[1] # We extract the relevant column
gdp2000 = pd.json_normalize(get_valid_country(data_2000))  # Filtering to keep only real countries and creating the DataFrame

## Here, we're curating our dataframe for GDP per capita in 2000
gdp2000["value"] = pd.to_numeric(gdp2000["value"], errors="coerce")
gdp2000 = gdp2000.dropna(subset=["value"])

## Finally, we select and rename the relevant columns
gdp2000 = gdp2000[["countryiso3code", "value"]].rename(columns={"value": "gdp_2000"})

## GPD per capita 2010

params_2010 = {
    "format": "json",
      "per_page": 20000,
        "date": "2010"}

resp_2010 = requests.get(url_gdp, params=params_2010) # Getting the response from the API
data_2010 = resp_2010.json()[1] # We extract the relevant column
gdp2010 = pd.json_normalize(get_valid_country(data_2010)) # Filtering to keep only real countries and creating the DataFrame

## Here, we're curating our dataframe for GDP per capita in 2010
gdp2010["value"] = pd.to_numeric(gdp2010["value"], errors="coerce")
gdp2010 = gdp2010.dropna(subset=["value"])

## Finally, we select and rename the relevant columns
gdp2010 = gdp2010[["countryiso3code", "value"]].rename(columns={"value": "gdp_2010"})

## GPD per capita 2022

params_2022 = {
    "format": "json", 
    "per_page": 20000, 
    "date": "2022"}
resp_2022 = requests.get(url_gdp, params=params_2022) # Getting the response from the API
data_2022 = resp_2022.json()[1] # We extract the relevant column
gdp2022 = pd.json_normalize(get_valid_country(data_2022)) # Filtering to keep only real countries and creating the DataFrame

## Here, we're curating our dataframe for GDP per capita in 2010
gdp2022["value"] = pd.to_numeric(gdp2022["value"], errors="coerce")
gdp2022 = gdp2022.dropna(subset=["value"])

## Finally, we select and rename the relevant columns
gdp2022 = gdp2022[["countryiso3code", "country.value", "value"]].rename(
    columns={"country.value": "country", "value": "gdp_2022"})


# Let's now combine the data and perform the required calculations

gdp = pd.merge(gdp2000, gdp2010, on="countryiso3code", how="inner")
gdp = pd.merge(gdp, gdp2022, on="countryiso3code", how="inner")

## Remove countries where starting GDP = 0 (curation step to avoid division by zero)
gdp = gdp[(gdp["gdp_2000"] > 0) & (gdp["gdp_2010"] > 0)]


# Now, let's compute the % growths
gdp["pct_00_22"] = 100 * (gdp["gdp_2022"] - gdp["gdp_2000"]) / gdp["gdp_2000"]
gdp["pct_10_22"] = 100 * (gdp["gdp_2022"] - gdp["gdp_2010"]) / gdp["gdp_2010"]

# Let's pull income group from country_metadata (from Milestone 1)

countries_df = pd.json_normalize(country_metadata)

income = countries_df[["id", "incomeLevel.value"]].rename(
    columns={"id": "countryiso3code", "incomeLevel.value": "income_group"}
)

gdp = pd.merge(gdp, income, on="countryiso3code", how="left")

# We keep only the required 5 income categories
valid_income_groups = [
    "Low income",
    "Lower middle income",
    "Middle income",
    "Upper middle income",
    "High income"
]
gdp = gdp[gdp["income_group"].isin(valid_income_groups)] # Filtering to keep only valid income groups

# Now, let's compute the average % growth by income group

avg_00_22 = (
    gdp.groupby("income_group")["pct_00_22"]
    .mean()
    .reset_index()
    .sort_values("pct_00_22", ascending=True)
)

avg_10_22 = (
    gdp.groupby("income_group")["pct_10_22"]
    .mean()
    .reset_index()
    .sort_values("pct_10_22", ascending=True)
)

print("Average % GDP per capita change 2000–2022 by income group:")
display(avg_00_22)

print("\nAverage % GDP per capita change 2010–2022 by income group:")
display(avg_10_22)

Average % GDP per capita change 2000–2022 by income group:


,income_group,pct_00_22
1,Low income,175.733022
0,High income,218.412210
2,Lower middle income,287.749816
3,Upper middle income,368.016566



Average % GDP per capita change 2010–2022 by income group:


,income_group,pct_10_22
1,Low income,22.779111
0,High income,37.354910
3,Upper middle income,43.585105
2,Lower middle income,55.445763


## **2.4. MILESTONE 4: TOP 5 COUNTRIES’ GDP INCREASE PER INCOME GROUP**

For each of the income groups included in **Milestone 3** and for the period **2010–2022**, list the **Top 5 countries** in terms of **% GDP per capita increase**, along with the value.

**NOTE**: Do not consider the countries for which you do not have data either in **2010** or **2022**, or both of them.

*Write your code solution for Milestone 4 in the next code cell in the notebook.*

---


In [5]:
# SOLUTION MILESTONE 4

# We create a loop to display the top 5 countries by income group for the period 2010-2022

for grp in sorted(gdp["income_group"].dropna().unique()):
    sub = gdp[gdp["income_group"] == grp]
    top5 = sub.sort_values("pct_10_22", ascending=False).head(5)
    print(f"\nIncome group: {grp}")
    display(top5[["country", "countryiso3code", "pct_10_22"]])



Income group: High income


,country,countryiso3code,pct_10_22
76,Guyana,GUY,290.973900
128,Nauru,NRU,173.002974
86,Ireland,IRL,116.178724
106,Lithuania,LTU,112.068003
141,Panama,PAN,108.035894



Income group: Low income


,country,countryiso3code,pct_10_22
165,"Somalia, Fed. Rep.",SOM,161.460619
41,"Congo, Dem. Rep.",COD,104.303159
152,Rwanda,RWA,64.324185
103,Liberia,LBR,51.267228
75,Guinea-Bissau,GNB,45.525851



Income group: Lower middle income


,country,countryiso3code,pct_10_22
14,Bangladesh,BGD,258.666322
180,Timor-Leste,TLS,188.200714
50,Djibouti,DJI,158.256777
196,Viet Nam,VNM,146.422995
31,Cambodia,KHM,144.155463



Income group: Upper middle income


,country,countryiso3code,pct_10_22
38,China,CHN,180.188329
120,Moldova,MDA,135.731193
7,Armenia,ARM,116.087801
67,Georgia,GEO,105.070131
115,Marshall Islands,MHL,104.261508


## **2.5. MILESTONE 5: CO₂ EMISSION PER CAPITA**

Retrieve the **most recent non-empty value** for the amount of:

> *Total greenhouse gas emissions excluding LULUCF per capita (t CO₂e/capita)*

for **all the countries**.

Then display the **30 countries with the highest emissions per capita** along with:

- their **emission value**, and  
- the **year** corresponding to that value.

**NOTE**: You cannot manually search the year and use it in your query for this milestone.

*Write your code solution for Milestone 5 in the next code cell in the notebook.*

In [6]:
# SOLUTION MILESTONE 5

# Procedure to get the latest GHG emissions per capita (excluding LULUCF)
url = "https://api.worldbank.org/v2/country/all/indicator/EN.GHG.ALL.PC.CE.AR5"
params = {
    "format": "json",
    "per_page": 20000,
    "MRNEV": 1   # This replaces the need for a specific date/year range to get the most recent values
}

response = requests.get(url, params=params) # Getting the response from the API
data = response.json() # Transforming the response to JSON format

df = pd.json_normalize(data[1]) # Here, we're selecting only the relevant columns and create the dataframe

# Let's filter only real countries with the get_valid_country function
df = pd.json_normalize(get_valid_country(data[1]))

# Here, we're curating our dataframe for GHG emissions per capita
df["value"] = pd.to_numeric(df["value"], errors="coerce")
df["date"] = pd.to_numeric(df["date"], errors="coerce")

# Here, we drop NaN values
df = df.dropna(subset=["value", "date"])

# Then, we sort by country and by year (descending)
df = df.sort_values(["countryiso3code", "date"], ascending=[True, False])

# Let's perform another curation step: We keep only the latest entry per country
latest = df.drop_duplicates(subset=["countryiso3code"], keep="first")

# Finally, we select and rename the relevant columns
latest = latest[["country.value", "countryiso3code", "date", "value"]]
latest = latest.rename(columns={
    "country.value": "country",
    "date": "year",
    "value": "ghg_per_capita"
})

# Let's now get the top 30 countries by latest GHG emissions per capita (excluding LULUCF)
top30 = latest.sort_values("ghg_per_capita", ascending=False).head(30)
print("Top 30 countries by latest GHG emissions per capita (excluding LULUCF):")
display(top30)



Top 30 countries by latest GHG emissions per capita (excluding LULUCF):


,country,countryiso3code,year,ghg_per_capita
142,Palau,PLW,2023,84.712585
151,Qatar,QAT,2023,58.125655
13,Bahrain,BHR,2023,40.413136
101,Kuwait,KWT,2023,34.597418
27,Brunei Darussalam,BRN,2023,26.490743
191,United Arab Emirates,ARE,2023,25.546505
140,Oman,OMN,2023,25.239891
183,Trinidad and Tobago,TTO,2023,24.999452
124,Mongolia,MNG,2023,24.045135
157,Saudi Arabia,SAU,2023,23.889996
